# Import des modules

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
from ast import literal_eval
import csv

# Création des variables

In [3]:
DF0 = pd.read_csv("./../Tables/ReferentielPrenoms.csv",converters={"FormesNonNormalisées": literal_eval}).set_index("PrénomNormalisé")
DF0

,PrénomFrançais,Types,FormesNonNormalisées
PrénomNormalisé,,,
Adalhelm,Alleaume,"['masc', 'germanique']","[Adalhelm, Adelhelm, Alhelm]"
Adam,Adam,"['masc', 'hébraïque']",[Adam]
Adolff,Adolphe,"['masc', 'germanique']","[Adolf, Adolff]"
Andres,André,"['masc', 'grec']",[Andres]
Anshelm,Anselme,"['masc', 'germanique']",[Anshelm]
...,...,...,...
Wetzel,NaN,"['masc', 'germanique']",[Wetzel]
Wilhelm,Guillaume,"['masc', 'germanique']",[Wilhelm]
Wirich,NaN,"['masc', 'germanique']","[Wirich, Wyrich]"


# Création de la soupe

In [3]:
soup = BeautifulSoup(open('./../Sources/EditionsXML/aves_4R_p131-220.xml', 'r'))

# Création de la liste des éléments de la soupe à comparer

In [4]:
ListePrénomDoc = []
for forename in soup.find_all('forename'):
    ListePrénomDoc.append(forename.get_text())
print (ListePrénomDoc)

['Johanns', 'Lütolt', 'Adam', 'Claus', 'Wilhelm', 'Wetzel', 'Heinrich', 'Thoman', 'Reimbolt', 'Hug', 'Otteman', 'Wernlin', 'Claus', 'Johanns', 'Hesseman', 'Claus', 'Johanns', 'Hug', 'Götze', 'Heintzman', 'Symont', 'Reimbolt', 'Wernher', 'Wernher', 'Hanman', 'Hig', 'Claus', 'Hegel', 'Cüneman', 'Hanman', 'Gerung', 'Wilhelm', 'Claus', 'Claus', 'Ülrich', 'Sigel', 'Erhart', 'Andres', 'Diebolt', 'Cüntz', 'Diethrich', 'Johanns', 'Leonhart', 'Johanns', 'Peter', 'Ülrich', 'Andres', 'Johanns', 'Heintzman', 'Ülrich', 'Claus', 'Hanns', 'Claus', 'Peter', 'Cüntz', 'Hug', 'Walther', 'Johanns', 'Ülrich', 'Bere', 'Behtolt', 'Peter', 'Leonhart', 'Burckart', 'Rülman', 'Dietrich', 'Reimbolt', 'Heinrich', 'Schochman', 'Bernhart', 'Ülrich', 'Hanns Rüdolff', 'Thoman', 'Johanns', 'Wilhelm', 'Reimbolt', 'Johanns', 'Hug', 'Johanns', 'Bernhart', 'Johanns', 'Johanns', 'Johanns', 'Walther', 'Rüdolff', 'Werlin', 'Rüllin', 'Johanns', 'Wilhelm', 'Hanseman', 'Johanns', 'Johanns', 'Heinrich', 'Johanns', 'Eberlin', 'Hei

# Comparaison de la liste de la soupe à notre dataframe

In [5]:
for element in ListePrénomDoc :
    
    #Création des listes de références à partir du dataframe DF1.
    Colonne_FormesNonNormalisées = DF0['FormesNonNormalisées'].tolist()
    Colonne_PrénomNormalisé = DF0.index.tolist()
    
    #Vérification de l'existence de la forme non normalisée dans la liste des listes de formes non-normalisées.
    CheckForenameNN = any(element in sublist for sublist in Colonne_FormesNonNormalisées)
    DicoPersonne = {}

    #Sinon, on demande un str en entrée pour le comparer à la liiste des formes normalisées.
    if CheckForenameNN != True :
        ForenameN = input('Entrez la forme normalisée du prénom ' + element + ' :')
        
        #Si l'entrée figure dans la liste des prénoms normalisées, on ajoute la forme non normalisée à la liste de celles-ci dans le dataframe.
        if ForenameN in Colonne_PrénomNormalisé:
            CelluleModif = DF0.loc[ForenameN, 'FormesNonNormalisées']
            #print(type(CelluleModif))
            #print(CelluleModif)
            CelluleModif.append(element)
            
        #Sinon, on crée un dataframe qu'on additionnera à l'existant.
        else :
            
            #On crée le couple 'PrénomNormalisé' dont la valeur correspond à la forme la plus courante du prénom.
            DicoPersonne['PrénomNormalisé'] = ForenameN
            
            #On crée le couple 'FormeFR' dont la valeur correspond à la traduction française du prénom.
            FormeFR = input('Entrez la traduction française du prénom ' + element +' (possibilité de laisser vide)')
            DicoPersonne['PrénomFrançais'] = FormeFR
            
            #On crée le couple 'Types' dont la valeur est une liste contenant des tags proposant une description typologique du prénom.
            typologie = []
            n = int(input('Entrez le nombre de types à attribuer au prénom ' + element + ' :'))
            for i in range(0,n):
                Type = input('Entrez un des types à attribuer au prénom ' + element + ' :')
                typologie.append(Type)
            DicoPersonne['Types'] = typologie
            
            #On crée le couple 'FormesNonNormalisées' dont la valeur est une liste contenant toutes les formes alternatives du prénom.
            ListeNN = []
            ListeNN.append(element)
            if element != ForenameN :
                ListeNN.append(ForenameN)
            DicoPersonne['FormesNonNormalisées'] = ListeNN
            
            #On crée le dataframe à ajouter à l'existant.
            DF1 = pd.DataFrame([DicoPersonne]).set_index('PrénomNormalisé')
            
            #On concatène le dictionnaire obtenu à notre Referentiel avant de recommencer toute l'opération.
            DF0 = pd.concat([DF0,DF1])
DF0

Entrez la forme normalisée du prénom Cuͦn :Kühne
Entrez la forme normalisée du prénom Hanns Balthazar :Johanns-Balthazar
Entrez la forme normalisée du prénom Hannß Conrat :Johanns-Konrat
Entrez la forme normalisée du prénom Hannsheinrich :Johanns-Heinrich
Entrez la forme normalisée du prénom Hannßconrat :Johanns-Konrat
Entrez la forme normalisée du prénom Goͤtze :Götze
Entrez la forme normalisée du prénom Sỳmont :Symont
Entrez la forme normalisée du prénom Churin :Khurin
Entrez la traduction française du prénom Churin (possibilité de laisser vide)
Entrez le nombre de types à attribuer au prénom Churin :2
Entrez un des types à attribuer au prénom Churin :masc
Entrez un des types à attribuer au prénom Churin :germanique
Entrez la forme normalisée du prénom Boͤldelin :Böldemann
Entrez la traduction française du prénom Boͤldelin (possibilité de laisser vide)
Entrez le nombre de types à attribuer au prénom Boͤldelin :2
Entrez un des types à attribuer au prénom Boͤldelin :masc
Entrez un des 

Entrez la traduction française du prénom Paulus (possibilité de laisser vide)Paul
Entrez le nombre de types à attribuer au prénom Paulus :2
Entrez un des types à attribuer au prénom Paulus :masc
Entrez un des types à attribuer au prénom Paulus :latin
Entrez la forme normalisée du prénom Rùlin :Ruhlemann
Entrez la forme normalisée du prénom Matheus :Matheus
Entrez la traduction française du prénom Matheus (possibilité de laisser vide)Matthieu
Entrez le nombre de types à attribuer au prénom Matheus :2
Entrez un des types à attribuer au prénom Matheus :masc
Entrez un des types à attribuer au prénom Matheus :latin
Entrez la forme normalisée du prénom Rülin :Ruhlemann
Entrez la forme normalisée du prénom Otte :Ottemann
Entrez la forme normalisée du prénom Hanswilhelm :Johanns-Wilhelm
Entrez la forme normalisée du prénom Ott :Ottemann
Entrez la forme normalisée du prénom Balthazar :Balthazar
Entrez la traduction française du prénom Balthazar (possibilité de laisser vide)Balthazar
Entrez le n

,PrénomFrançais,Types,FormesNonNormalisées
PrénomNormalisé,,,
Adalhelm,Alleaume,"['masc', 'germanique']","[Adalhelm, Adelhelm, Alhelm]"
Adam,Adam,"['masc', 'hébraïque']",[Adam]
Adolff,Adolphe,"['masc', 'germanique']","[Adolf, Adolff]"
Andres,André,"['masc', 'grec']",[Andres]
Anshelm,Anselme,"['masc', 'germanique']",[Anshelm]
...,...,...,...
Eucharius,Euchaire,"[masc, latin]",[Eucharius]
Lamprecht,,"[masc, germanique]","[Lampreht, Lamprecht]"
Jeronimus,Jérôme,"[masc, latin]",[Jeronimus]


In [7]:
DF0.to_csv("./../Tables/RéférentielPrénoms.csv")